# Instructions
The following code was designed in order to score motion/freezing data for a single video file. It allows for cropping of the video frame to reduce the influence of tethers. Motion is detected by capturing the number of pixels whose frame by frame grayscale change exceeds a user-defined motion threshold.  Freezing is then assessed by calculating when motion drops below a user-defined freezing threshold.  The user is able to visualize raw video, animal motion, and whether an animal is deemed to be freezing in order optimize parameters.  Final output is a csv that provides frame-by-frame motion and freezing across the session.  Additionally, a summary file can be generated that allows the user to get motion/freezing for specified time bins.  With the exception of setting the file path and a small number of parameters, the user can run through code.  

***Once parameters are found that work well, batch processing of multiple videos can be performed using FreezeAnalysis_BatchProcess.ipynb.***

---
# 1. Load Necessary Packages
The following code loads neccessary packages and need not be changed by the user.

In [49]:
%load_ext autoreload
%autoreload 2
import os
import holoviews as hv
import numpy as np
import pandas as pd
import FreezeAnalysis_Functions as fz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
# 2. User Sets Directory and File Information

`dpath` : The directory path of the folder containing the video to be processed. Note that if you are using a Windows path with backslashes, place an ‘r’ in front of the directory path to avoid an error (e.g., r’\Users\DeniseCaiLab\Videos’).

`file` : The filename of the video, including the file extension.

`start` : The frame of the video on which to begin processing.  0 is the first frame.  By knowing the video’s frame rate (e.g., 30 frames per second), the user can start processing the video at a specific timepoint.  For instance, to begin processing 20 seconds into the video, once the animal has been placed into the arena, one could enter 600 if the frame rate were 30 frames per second.  If you are uncertain of your video’s frame rate, this information will be printed by ezTrack when Cell 3 is run.

`end` : The frame of the video to end processing on.  If the user would like to process from the start frame to the end of the video, this can be set to None.

`dsmpl` : The amount to down-sample each frame.  If processing is going slow, down-sampling can help. A value of 1 indicates no down-sampling, while a value of 0.25 indicates that the frame will be down-sampled to ¼ the original size.  Note that if down-sampling is performed, all pixel coordinate output will be in the dimensions of the down-sampled video.

`stretch` : Allows the user to alter the aspect ratio of the presented output.  This is useful when videos have irregular dimensions and are difficult to see (e.g., an aspect ratio of 1:100).  The width/height will be scaled by the factor provided. Note that this only affects the appearance of visualizations and does not modify the video or the interpretation of the output.

***Processing going slow?  Consider downsampling!***  Often times tracking does not not require 1080p or whatever high def resolution videos are sometimes acquired using. Try setting `dsmpl` to something lower than 1 to implement downsampling (1=no downsampling). For example, if set to 0.25, each frame will be downsampled to 1/4 the original size.  Note that the same downsampling value should be used during calibration.

In [52]:
video_dict = {
    'dpath'   : "/media/rory/Padlock_DT/Fear_Conditioning_Control/NewVideos/Conditioning/",  
    'file'    : 'RRD276_C18457_Conditioning.avi',
    'start'   : 0, 
    'end'     : 32396,
    'dsmpl'   : 1,
    'stretch' : dict(width=1, height=1)
}

In [10]:
import cv2

cap = cv2.VideoCapture("/media/rory/Padlock_DT/Fear Conditioning Control/NewVideos/Conditioning/RRD276_C18457_Conditioning.avi")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)

32396


---
# 3. Load Video and Crop Frame if Desired
To crop video frame, after running code below, select box selection tool beneath image (square with a plus sign).  To start drawing region to be included in analyis, double click image.  Double click again to finalize region.  If you decide to change region, it is best to rerun this cell and subsequent steps.

If the size of the image is too small/large, alter the first line of code.  100 is the standard size.  200 will produce an image 2x the size, and so on.

In [53]:
%%output size=100

img_crp, video_dict = fz.LoadAndCrop(video_dict, cropmethod="Box")
img_crp

file: /media/rory/Padlock_DT/Fear_Conditioning_Control/NewVideos/Conditioning/RRD276_C18457_Conditioning.avi
total frames: 32396
nominal fps: 60.0
dimensions (h x w): 240,320


:Overlay
   .Image.I    :Image   [x,y]   (z)
   .Polygons.I :Polygons   [x,y]

---
# 4. Analyze Motion Across Session

### 4a. User Sets Motion Threshold
Below, user sets `mt_cutoff`, the grayscale difference value required for a pixel to be counted as changing from one frame to the next. Can be determined using FreezeAnalysis_Calibration.ipynb.

In [54]:
mt_cutoff = 8.73573982600555

### 4b. Detect Motion and Plot
Here, code loops through all frames and detects number of pixels whose grayscale change exceeds `mt_cutoff` per frame.  In addition to interactive plot options, one can also change the plot size by changing h and w in code below (e.g `h,w = 300,2000`)

In [55]:
%%output size=100

h,w = 300,1000 

Motion = fz.Measure_Motion(video_dict, mt_cutoff, SIGMA=1)  
print(len(Motion))
plt_mt = hv.Curve((np.arange(len(Motion)),Motion),'Frame','Pixel Change').opts(
    height=h,width=w,line_width=1,color="steelblue",title="Motion Across Session")
plt_mt

cap_max: 32396
length of Motion: 32396
range with len(Motion) 32395


 50%|████▉     | 16197/32395 [00:19<00:19, 823.99it/s]


no. frames detected: 16197
no. frames not detected: 1
total frames processed: 16197

16197


:Curve   [Frame]   (Pixel Change)

---
# 5. Analyze Session Freezing

### 5a. User Selects Freezing Parameters
Below user sets `FreezeThresh`, the upper bound in frame-by-frame pixels changed for freezing to be detected, and `MinDuration`, the number of frames motion must be below FreezeThresh to begin accruing freezing.

In [56]:
FreezeThresh = 200 
MinDuration = 15 

### 5b. Measure Freezing and Save
Freezing is calculated with cell below.  In addition to interactive plot options, one can also change the plot size by changing h and w in code below (e.g `h,w = 300,2000`)

In [57]:
%%output size=100

h,w = 300,1000 

Freezing = fz.Measure_Freezing(Motion,FreezeThresh,MinDuration)  
fz.SaveData(video_dict,Motion,Freezing,mt_cutoff,FreezeThresh,MinDuration)
print('Average Freezing: {x}%'.format(x=np.average(Freezing)))

plt_fz = hv.Area(Freezing*(Motion.max()/100),'Frame','Motion').opts(
    color='lightgray',line_width=0,line_alpha=0)
plt_mt = hv.Curve((np.arange(len(Motion)),Motion),'Frame','Motion').opts(
    height=h,width=w,line_width=1, color='steelblue',
    title="Motion Across Session with Freezing Highlighted in Gray")
plt_fz*plt_mt*hv.HLine(y=FreezeThresh).opts(color='red',line_width=1,line_dash='dashed')

Average Freezing: 27.363091930604433%


:Overlay
   .Area.I  :Area   [Frame]   (Motion)
   .Curve.I :Curve   [Frame]   (Motion)
   .HLine.I :HLine   [x,y]

In [46]:
print(len(list(Freezing)))
print(len(list(Motion)))

16197
16197


Example: A video is 8:59, so 539s, which equals to 32,340 frames, 

---
# 6. (Optional) Display Section of Video with Motion and Freezing
***After analyzing freezing*** a section of the video can be replayed and the animal's state - "Active/Freezing" - will be displayed.  ). If the animal is freezing but said to be active, try increasing `FreezeThresh`.  If the animal is moving and said to be freezing, try decreasing FreezeThresh. When mt_cutoff is too low, pixel fluctuations not associated with the animal’s movement will be visible.  Alternatively, when mt_cutoff is too high, animal motion will be invisible or very low.

`start` : The frame video playback is to be started on. Note that this is relative to the start of tracking, where 0 is the first tracked frame (the defined start frame from Cell 2)

`stop` : The frame video playback is to end on.  Note that this is relative to the start of tracking, where 0 is the first tracked frame (the defined start frame from Cell 2)

`fps` : The speed of video playback.  Must be an integer.  Video playback may also be slower depending upon computer speed.  This is because fps sets the time imposed between presented frames but cannot control the duration of time it will take a user’s computer to present each frame.

`resize` : If the user wants the output to be larger or smaller, or they want the aspect ratio to be different, resize can be supplied as in the following example: `'resize’ : (100,200)`. Here, the first number corresponds to the adjusted width of the frame, whereas the second number corresponds to the adjusted height.  Both numbers reflect pixel units and should be integers. Set resize equal to None if no resizing is to be done.

`save_video` : If the user would like to save the video clip, save_video can be set to True.

In [58]:
print(cv2.__version__)

4.5.3


In [66]:
display_dict = {
    'start'      : 1, 
    'end'        : 10000,
    'fps'        : 30,
    'resize'     : None,
    'save_video' : True
}

fz.PlayVideo(video_dict,display_dict,Freezing,mt_cutoff,SIGMA=1)

Done playing segment


---
# 7. (Optional) Create Binned Summary Report and Save
The code below allows the user to either save a csv containing summary data for user-defined bins (e.g., % time freezing for each minute) or a session-wide average. 

***If you only want a session average***, set `bin_dict = None`

**To specify bins**, set `bin_dict` using the following notation, where start and stop represent time ***in frames***.


```
bin_dict = {
    'BinName1': (start, stop),
    'BinName2': (start, stop),
    'BinName3': (start, stop),
}
```

In [ ]:
bin_dict = {
    '1' : (0,10),
    '2' : (10,20),
    '3' : (20,30)
}

summary = fz.Summarize(video_dict,Motion,Freezing,FreezeThresh,MinDuration,mt_cutoff,bin_dict=bin_dict)
summary.to_csv(os.path.splitext(video_dict['fpath'])[0] + '_SummaryStats.csv', index=False)
summary